Tomado de [Sharone Li](https://towardsdatascience.com/concatenate-multiple-and-messy-dataframes-efficiently-80847b4da12b)

In [8]:
import pandas as pd


In [19]:
import glob
import numpy as np
import os
import pandas as pd
all_files = glob.glob("/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/2*.csv")

In [20]:
all_files

['/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/2019.csv',
 '/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/2020.csv',
 '/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/2018.csv',
 '/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/2022.csv',
 '/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/2021.csv',
 '/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/2016.csv',
 '/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/2017.csv',
 '/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/2015.csv']

In [18]:
import pathlib
col_names=[]

for txt_file in pathlib.Path("/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/").glob('2*.csv'):
    df.columns=[col.lower() for col in list(df.columns)]
    col_names.append(df.columns.values.tolist())
      
col_names

[['formatted date',
  'summary',
  'precip type',
  'temperature (c)',
  'apparent temperature (c)',
  'humidity',
  'wind speed (km/h)',
  'wind bearing (degrees)',
  'visibility (km)',
  'loud cover',
  'pressure (millibars)',
  'daily summary'],
 ['formatted date',
  'summary',
  'precip type',
  'temperature (c)',
  'apparent temperature (c)',
  'humidity',
  'wind speed (km/h)',
  'wind bearing (degrees)',
  'visibility (km)',
  'loud cover',
  'pressure (millibars)',
  'daily summary'],
 ['formatted date',
  'summary',
  'precip type',
  'temperature (c)',
  'apparent temperature (c)',
  'humidity',
  'wind speed (km/h)',
  'wind bearing (degrees)',
  'visibility (km)',
  'loud cover',
  'pressure (millibars)',
  'daily summary'],
 ['formatted date',
  'summary',
  'precip type',
  'temperature (c)',
  'apparent temperature (c)',
  'humidity',
  'wind speed (km/h)',
  'wind bearing (degrees)',
  'visibility (km)',
  'loud cover',
  'pressure (millibars)',
  'daily summary'],
 ['f

Next, let’s use for loop to read all the files into pandas dataframes. If these datasets all have the same column names and the columns are in the same order, we can easily concatenate them using pd.concat(). Let’s check if this is the case using the following code (notice that in line 4 I changed all the column names to lower-case for the convenience of checking).

In [23]:
dfs=[]
for i,filename in enumerate(all_files, start=0):
    df=pd.read_csv(filename)
    df.columns=[col.lower() for col in list(df.columns)]
    country_col = [col for col in df.columns if 'country' in col]
    score_col=[col for col in df.columns if any(str == col for str in ['score','happiness_score', 'happiness.score', 'happiness score','ladder score'])]
    
    df1= pd.concat([df[country_col], df[score_col]],axis=1)
    year_col=[*range(2015, 2023, 1)]
    df1['year']=year_col[i]
    
    dfs.append(df1)
    dfs[i].columns = ['country','happiness_score','year']

Line 1: We create an empty list dfs. We will later append each dataframe to this list with for loop iterations.
Line 6–7: we iterate and extract the ‘country’ column and the ‘happiness score’ column from each dataframe based on certain patterns observed from their column names.
Line 9–11: we create a new dataframe df1 that only has the three columns we need — country_col, score_col, and year_col.
Line 13: we append each of the 8 pre-processed dataframes to the list dfs with for loop iterations.
Line 14: we assign the same column names to all the dataframes in the list so that we can concatenate them in the next step which requires identical column names.

In [22]:
dfs[0].head()

,country,happiness_score,year
0,Finland,7.769,2015
1,Denmark,7.600,2015
2,Norway,7.554,2015
3,Iceland,7.494,2015
4,Netherlands,7.488,2015


Finally, with all the dataframes having the same column names and order, we can concatenate them easily using the pd.concat() method. In line 1, we concatenate all the dataframes in the list of dfs, and in line 2–3 we clean the ‘happiness score’ and ‘country’ columns to make sure our data is ready for visualization in the next section.

In [35]:
df_all = pd.concat([dfs[i] for i in range(8)], axis=0)
#df_all['happiness_score']=np.where(df_all['year']==2022, df_all['happiness_score'].str.replace(',','.'),df_all['happiness_score'])
df_all['country'] = df_all['country'].str.replace(r'*', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
import plotly.express as px

df_ISO=pd.read_csv('/content/drive/MyDrive/CATEDRA/ANALISISGEOESPACIAL/AnalisisGeoespacial/data/countries_continents_codes_flags_url.csv')
df_final = pd.merge(df_all, df_ISO, how="left", on=["country"])
df_final=df_final[['country','happiness_score','year','alpha-3']]
df_final.dropna(inplace=True)

fig = px.choropleth(df_final,
                    locations='alpha-3', 
                    color='happiness_score',
                    color_continuous_scale="rdylgn", 
                    animation_frame='year'
                    )

fig.update_layout(
      title_text = 'World Happiness (2015-2022)',
      title_font_family="Times New Roman",
      title_font_size = 22,
      title_font_color="black", 
      title_x=0.46, 
         )

fig.show()